In [ ]:
csv_path = r'C:\Users\Admin\OneDrive\Desktop\annot.csv'
image_folder = r'C:\Users\Admin\OneDrive\Desktop\har1'
resized_image_folder = r'C:\Users\Admin\OneDrive\Desktop\resized_images'
output_dir = r'C:\Users\Admin\OneDrive\Desktop\anno_xml'

# image data collection (image pixel values)

In [1]:
import os
from PIL import Image

image_folder = r'C:\Users\Admin\OneDrive\Desktop\resized_images'
image_info = {}

for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        image_path = os.path.join(image_folder, filename)
        try:
            with Image.open(image_path) as img:
                width, height = img.size
                image_info[filename] = {'width': width, 'height': height}
        except IOError:
            print(f"Unable to open image: {filename}")


image_count = len(image_info)
print(f"Number of items in image_info: {image_count}")

Number of items in image_info: 425


In [2]:
# Assuming you already have the `image_info` dictionary
first_image = next(iter(image_info.items()), None)

if first_image is not None:
    image_name, info = first_image
    print(f"Image Name: {image_name}")
    print(f"Width: {info['width']}")
    print(f"Height: {info['height']}")
else:
    print("No images found in image_info dictionary.")


Image Name: 0000e8b36676338b.jpg
Width: 224
Height: 224


In [3]:
import os
from PIL import Image

image_folder = r'C:\Users\Admin\OneDrive\Desktop\har_img_2'
image_info_1 = {}

for filename in os.listdir(image_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
        image_path = os.path.join(image_folder, filename)
        try:
            with Image.open(image_path) as img:
                width, height = img.size
                image_info_1[filename] = {'width': width, 'height': height}
        except IOError:
            print(f"Unable to open image: {filename}")


image_count = len(image_info_1)
print(f"Number of items in image_info: {image_count}")


Number of items in image_info: 425


In [4]:
# Assuming you already have the `image_info` dictionary
first_image = next(iter(image_info_1.items()), None)

if first_image is not None:
    image_name, info = first_image
    print(f"Image Name: {image_name}")
    print(f"Width: {info['width']}")
    print(f"Height: {info['height']}")
else:
    print("No images found in image_info dictionary.")


Image Name: 0000e8b36676338b.jpg
Width: 1024
Height: 685


# reading xml files

In [5]:
import xml.etree.ElementTree as ET
import os

def read_xml_folder(xml_folder):
    annotations = {}

    xml_files = os.listdir(xml_folder)
    for xml_file in xml_files:
        xml_path = os.path.join(xml_folder, xml_file)
        if os.path.isfile(xml_path):
            tree = ET.parse(xml_path)
            root = tree.getroot()

            image_filename = root.find('filename').text

            if image_filename not in annotations:
                annotations[image_filename] = []

            for object_elem in root.findall('object'):
                label = object_elem.find('name').text

                bndbox_elem = object_elem.find('bndbox')
                xmin = float(bndbox_elem.find('xmin').text)
                ymin = float(bndbox_elem.find('ymin').text)
                xmax = float(bndbox_elem.find('xmax').text)
                ymax = float(bndbox_elem.find('ymax').text)

                annotations[image_filename].append({
                    'label': label,
                    'xmin': xmin,
                    'ymin': ymin,
                    'xmax': xmax,
                    'ymax': ymax
                })

    return annotations

# Example usage
xml_folder = r'C:\Users\Admin\OneDrive\Desktop\anno_xml'

annotations = read_xml_folder(xml_folder)


In [6]:
print("Count of annotation items:", sum(len(boxes) for boxes in annotations.values()))


Count of annotation items: 16433


In [ ]:
print(annotations)

In [7]:
#printing the first item from annotations.
# Assuming you already have the 'annotations' dictionary
first_image_filename = next(iter(annotations.keys()))
first_annotations = annotations[first_image_filename]

print("First Image Filename:", first_image_filename)
print("First Annotations:")
for annotation in first_annotations:
    print(annotation)


First Image Filename: 0000e8b36676338b.jpg
First Annotations:
{'label': 'text', 'xmin': 308.0, 'ymin': 135.8, 'xmax': 353.96, 'ymax': 153.13}
{'label': 'text', 'xmin': 383.8, 'ymin': 140.15, 'xmax': 454.69, 'ymax': 161.55}
{'label': 'text', 'xmin': 137.87, 'ymin': 463.48, 'xmax': 172.13, 'ymax': 489.28000000000003}
{'label': 'text', 'xmin': 174.15, 'ymin': 471.14, 'xmax': 209.22, 'ymax': 497.34}
{'label': 'text', 'xmin': 637.92, 'ymin': 456.18, 'xmax': 698.5799999999999, 'ymax': 474.94}


# grouping 

In [10]:
#grouping andd displaying the first the same for 1st item

grouped_annotations = {}

for image_filename, boxes in annotations.items():
    if image_filename not in grouped_annotations:
        grouped_annotations[image_filename] = {
            'boxes': [],
            'labels': []
        }
    
    for box in boxes:
        grouped_annotations[image_filename]['boxes'].append([box['xmin'], box['ymin'], box['xmax'], box['ymax']])
        grouped_annotations[image_filename]['labels'].append(box['label'])

# Example usage
first_image_filename = next(iter(grouped_annotations.keys()))
print("Image Filename:", first_image_filename)
print("Bounding Boxes:", grouped_annotations[first_image_filename]['boxes'])
print("Class Labels:", grouped_annotations[first_image_filename]['labels'])


Image Filename: 0000e8b36676338b.jpg
Bounding Boxes: [[308.0, 135.8, 353.96, 153.13], [383.8, 140.15, 454.69, 161.55], [137.87, 463.48, 172.13, 489.28000000000003], [174.15, 471.14, 209.22, 497.34], [637.92, 456.18, 698.5799999999999, 474.94]]
Class Labels: ['text', 'text', 'text', 'text', 'text']


# normalizing bbox relative to image dimensions

In [11]:
import xml.etree.ElementTree as ET
import os
import cv2

def populate_grouped_annotations_and_image_info(xml_folder_path):
    grouped_annotations = {}
   

    for filename in os.listdir(xml_folder_path):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(xml_folder_path, filename)
            tree = ET.parse(xml_file_path)
            root = tree.getroot()

            # Extract image filename
            image_filename = root.find("filename").text

            # Extract image width and height from the image file
            image_path = os.path.join(image_folder, image_filename)
            image_width, image_height, _ = cv2.imread(image_path).shape

            # Process bounding boxes
            annotations = []
            for obj in root.findall("object"):
                label = obj.find("name").text
                bbox = obj.find("bndbox")
                xmin = float(bbox.find("xmin").text)
                ymin = float(bbox.find("ymin").text)
                xmax = float(bbox.find("xmax").text)
                ymax = float(bbox.find("ymax").text)

                # Normalize the bounding box coordinates
                normalized_xmin = xmin / image_width
                normalized_ymin = ymin / image_height
                normalized_xmax = xmax / image_width
                normalized_ymax = ymax / image_height

                # Append the normalized bounding box coordinates and label
                annotations.append({
                    'label': label,
                    'bbox': [normalized_xmin, normalized_ymin, normalized_xmax, normalized_ymax]
                })

            # Update grouped_annotations
            if image_filename not in grouped_annotations:
                grouped_annotations[image_filename] = {'annotations': annotations}
            else:
                grouped_annotations[image_filename]['annotations'].extend(annotations)

    return grouped_annotations

# Usage
xml_folder_path = r'C:\Users\Admin\OneDrive\Desktop\anno_xml'
grouped_annotations = populate_grouped_annotations_and_image_info(xml_folder_path)


In [ ]:
print(grouped_annotations)

In [ ]:
print(grouped_annotations.keys())


In [12]:
print(len(grouped_annotations))

425


In [13]:
# Printing the first item from grouped_annotations
first_image_filename = next(iter(grouped_annotations.keys()))
first_annotations = grouped_annotations[first_image_filename]['annotations']

print("Image Filename:", first_image_filename)
print("Bounding Boxes:")
for annotation in first_annotations:
    label = annotation['label']
    bbox = annotation['bbox']
    print("- Label:", label)
    print("  Bounding Box:", bbox)


Image Filename: 0000e8b36676338b.jpg
Bounding Boxes:
- Label: text
  Bounding Box: [0.44963503649635034, 0.1326171875, 0.5167299270072993, 0.149541015625]
- Label: text
  Bounding Box: [0.5602919708029197, 0.136865234375, 0.6637810218978102, 0.157763671875]
- Label: text
  Bounding Box: [0.20127007299270075, 0.4526171875, 0.2512846715328467, 0.47781250000000003]
- Label: text
  Bounding Box: [0.25423357664233576, 0.46009765625, 0.30543065693430654, 0.48568359375]
- Label: text
  Bounding Box: [0.9312700729927007, 0.44548828125, 1.019824817518248, 0.46380859375]


In [14]:
image_filename = '0000e8b36676338b.jpg' # Replace with the actual image filename
if image_filename in grouped_annotations:
    annotations1 = grouped_annotations[image_filename]['annotations']
    print(annotations1)
else:
    print("Image annotations not found.")


[{'label': 'text', 'bbox': [0.44963503649635034, 0.1326171875, 0.5167299270072993, 0.149541015625]}, {'label': 'text', 'bbox': [0.5602919708029197, 0.136865234375, 0.6637810218978102, 0.157763671875]}, {'label': 'text', 'bbox': [0.20127007299270075, 0.4526171875, 0.2512846715328467, 0.47781250000000003]}, {'label': 'text', 'bbox': [0.25423357664233576, 0.46009765625, 0.30543065693430654, 0.48568359375]}, {'label': 'text', 'bbox': [0.9312700729927007, 0.44548828125, 1.019824817518248, 0.46380859375]}]


# To convert the grouped annotations to the TFRecord format

In [15]:
import tensorflow as tf
import os
import xml.etree.ElementTree as ET

def create_tf_example(image_path, annotations, image_info):
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()
        
    image = tf.image.decode_jpeg(encoded_image)
    image_height, image_width, _ = image.shape
    
    
    xmins = []
    ymins = []
    xmaxs = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for annotation in annotations:
        label = annotation['label']
        bbox = annotation['bbox']
        
        xmins.append(bbox[0])
        ymins.append(bbox[1])
        xmaxs.append(bbox[2])
        ymaxs.append(bbox[3])
        classes_text.append(label.encode('utf8'))
        classes.append(2)  # Assuming there is two class 'text' & 'non-text'
        
        
        
    feature_dict = {
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_info['height']])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[image_info['width']])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[os.path.basename(image_path).encode('utf8')])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[os.path.basename(image_path).encode('utf8')])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=['jpg'.encode('utf8')])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }


    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example


def convert_annotations_to_tfrecord(image_folder_path, grouped_annotations, image_info, output_path):
    writer = tf.io.TFRecordWriter(output_path)

    for image_filename, annotation in grouped_annotations.items():
        image_path = os.path.join(image_folder_path, image_filename)
        annotations = annotation['annotations']
        tf_example = create_tf_example(image_path, annotations, image_info[image_filename])
        writer.write(tf_example.SerializeToString())

    writer.close()
    print(f"TFRecord saved to {output_path}")
    
    
# Usage
image_folder_path = r'C:\Users\Admin\OneDrive\Desktop\har_img_2'
xml_folder_path = r'C:\Users\Admin\OneDrive\Desktop\anno_xml'
#grouped_annotations = populate_grouped_annotations_and_image_info(xml_folder_path)
output_path = r'C:\Users\Admin\OneDrive\Desktop\tfrecord_0.tfrecord'

convert_annotations_to_tfrecord(image_folder_path, grouped_annotations, image_info_1, output_path)


TFRecord saved to C:\Users\Admin\OneDrive\Desktop\tfrecord_0.tfrecord


# This code will read each serialized example from the TFRecord file and parse it using the parse_tfrecord() function. Then print the parsed example to inspect its contents.

In [ ]:
import tensorflow as tf

def parse_tfrecord(serialized_example):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)
    return example

# Path to the TFRecord file
tfrecord_path = r'C:\Users\Admin\OneDrive\Desktop\tfrecord_0.tfrecord'

# Read and print the contents of the TFRecord file
raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
for raw_record in raw_dataset:
    example = parse_tfrecord(raw_record)
    print(example)


# printing TFRecord data 1 content

In [19]:
import tensorflow as tf

def print_tfrecord_data(tfrecord_path):
    dataset = tf.data.TFRecordDataset(tfrecord_path)

    # Define the feature description
    feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

    # Parse the example records
    def _parse_function(example_proto):
        return tf.io.parse_single_example(example_proto, feature_description)

    # Apply parsing function to the dataset
    parsed_dataset = dataset.map(_parse_function)

    # Print the data from the first record
    for record in parsed_dataset.take(1):
        for key, value in record.items():
            print(f'{key}: {value}')

# Specify the path to the TFRecord file
tfrecord_path = r'C:\Users\Admin\OneDrive\Desktop\tfrecord_0.tfrecord'

# Call the function to print the TFRecord data
print_tfrecord_data(tfrecord_path)


image/object/bbox/xmax: SparseTensor(indices=tf.Tensor(
[[0]
 [1]
 [2]
 [3]
 [4]], shape=(5, 1), dtype=int64), values=tf.Tensor([0.51672995 0.66378105 0.25128466 0.30543065 1.0198249 ], shape=(5,), dtype=float32), dense_shape=tf.Tensor([5], shape=(1,), dtype=int64))
image/object/bbox/xmin: SparseTensor(indices=tf.Tensor(
[[0]
 [1]
 [2]
 [3]
 [4]], shape=(5, 1), dtype=int64), values=tf.Tensor([0.44963503 0.56029195 0.20127007 0.25423357 0.93127006], shape=(5,), dtype=float32), dense_shape=tf.Tensor([5], shape=(1,), dtype=int64))
image/object/bbox/ymax: SparseTensor(indices=tf.Tensor(
[[0]
 [1]
 [2]
 [3]
 [4]], shape=(5, 1), dtype=int64), values=tf.Tensor([0.14954102 0.15776367 0.4778125  0.4856836  0.4638086 ], shape=(5,), dtype=float32), dense_shape=tf.Tensor([5], shape=(1,), dtype=int64))
image/object/bbox/ymin: SparseTensor(indices=tf.Tensor(
[[0]
 [1]
 [2]
 [3]
 [4]], shape=(5, 1), dtype=int64), values=tf.Tensor([0.13261719 0.13686523 0.4526172  0.46009767 0.44548827], shape=(5,), d

# printing the number of records in the .tfrecord file

In [20]:
import tensorflow as tf

def count_tfrecord_records(tfrecord_path):
    dataset = tf.data.TFRecordDataset(tfrecord_path)

    # Count the number of records in the dataset
    num_records = 0
    for _ in dataset:
        num_records += 1

    return num_records

# Specify the path to the TFRecord file
tfrecord_path = r'C:\Users\Admin\OneDrive\Desktop\tfrecord_0.tfrecord'

# Call the function to count the number of records
num_records = count_tfrecord_records(tfrecord_path)

# Print the number of records
print(f"Number of records: {num_records}")


Number of records: 425


# splitting data#

In [21]:
import random

def split_dataset(input_path, train_output_path, test_output_path, train_ratio=0.8):
    # Read the TFRecord file
    dataset = tf.data.TFRecordDataset(input_path)

    # Get the total number of examples
    num_examples = sum(1 for _ in dataset)

    # Calculate the number of examples for training and testing
    num_train_examples = int(425 * 0.8)
    num_test_examples = num_examples - num_train_examples

    # Shuffle the dataset
    dataset = dataset.shuffle(num_examples, reshuffle_each_iteration=False)

    # Split the dataset into training and testing datasets
    train_dataset = dataset.take(num_train_examples)
    test_dataset = dataset.skip(num_train_examples)

    # Save the training dataset to a TFRecord file
    writer = tf.data.experimental.TFRecordWriter(train_output_path)
    writer.write(train_dataset)

    # Save the testing dataset to a TFRecord file
    writer = tf.data.experimental.TFRecordWriter(test_output_path)
    writer.write(test_dataset)

    print("Datasets split and saved successfully.")

    # Print the number of items in the test and train datasets
    print("Number of items in train dataset:", num_train_examples)
    print("Number of items in test dataset:", num_test_examples)

# Usage
input_path = r'C:\Users\Admin\OneDrive\Desktop\tfrecord_0.tfrecord'
train_output_path = r'C:\Users\Admin\OneDrive\Desktop\train_dataset.tfrecord'
test_output_path = r'C:\Users\Admin\OneDrive\Desktop\test_dataset.tfrecord'
split_dataset(input_path, train_output_path, test_output_path)


Instructions for updating:
To write TFRecords to disk, use `tf.io.TFRecordWriter`. To save and load the contents of a dataset, use `tf.data.experimental.save` and `tf.data.experimental.load`
Datasets split and saved successfully.
Number of items in train dataset: 340
Number of items in test dataset: 85


# preprocessing def. - resizing images to (224,224)

In [ ]:
# import tensorflow as tf

def preprocess_dataset(dataset, target_size):
    def parse_example(example):
        feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

        example = tf.io.parse_single_example(example, feature_description)

        image = tf.io.decode_jpeg(example['image/encoded'], channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        mask = tf.io.decode_png(example['mask'], channels=1)

        target_size = (224, 224)
        image = tf.image.resize(image, target_size)
        mask = tf.image.resize(mask, target_size)

        bbox_xmin = tf.sparse.to_dense(example['bbox_xmin'])
        bbox_ymin = tf.sparse.to_dense(example['bbox_ymin'])
        bbox_xmax = tf.sparse.to_dense(example['bbox_xmax'])
        bbox_ymax = tf.sparse.to_dense(example['bbox_ymax'])
        labels = tf.sparse.to_dense(example['label'], default_value='')

        return image, mask, bbox_xmin, bbox_ymin, bbox_xmax, bbox_ymax, labels


    dataset = dataset.map(parse_example)

    return dataset


# preprocessing datasets (train & test)

In [ ]:
# import tensorflow as tf

def load_dataset(dataset_path):
    dataset = tf.data.TFRecordDataset(dataset_path)
    return dataset


# Load and preprocess train dataset
train_output_path = r'C:\Users\Admin\OneDrive\Desktop\train_dataset.tfrecord'
train_dataset = load_dataset(train_output_path)
train_dataset = preprocess_dataset(train_dataset, target_size)


# Load and preprocess test dataset
test_output_path = r'C:\Users\Admin\OneDrive\Desktop\test_dataset.tfrecord'
test_dataset = load_dataset(test_output_path)
test_dataset = preprocess_dataset(test_dataset, target_size)


# model

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Input, Reshape
from tensorflow.keras.models import Model

# Step 1: Define the model architecture
def build_mask_rcnn_model(num_classes):
    # Use ResNet50 as the backbone
    backbone = ResNet50(include_top=False, weights='imagenet', input_shape=(None, None, 3))

    # Add additional layers for mask prediction
    roi_pooling = tf.keras.layers.GlobalAveragePooling2D()
    mask_head = Conv2DTranspose(num_classes, (2, 2), strides=2, activation='softmax')

    # Define the input and output layers of the model
    input_layer = Input(shape=(None, None, 3))
    backbone_output = backbone(input_layer)
    roi_output = roi_pooling(backbone_output)
    reshaped_output = Reshape((1, 1, -1))(roi_output)  # Reshape the output for compatibility with mask_head
    mask_output = mask_head(reshaped_output)

    # Create the Mask R-CNN model
    model = Model(inputs=input_layer, outputs=mask_output)

    return model

# Step 2: Create the training pipeline
def create_training_pipeline(model, learning_rate):
    # Define your training pipeline
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss_fn)

# Step 3: Train the model
def train_model(model, train_dataset, epochs):
    # Unpack the train_dataset into inputs and targets
    inputs = []
    targets = []
    for image, mask, bbox_xmin, bbox_ymin, bbox_xmax, bbox_ymax, labels in train_dataset:
        inputs.append(image)
        targets.append(mask)
    inputs = tf.stack(inputs)
    targets = tf.stack(targets)

    # Train your model
    model.fit(inputs, targets, epochs=epochs)


# Step 4: Evaluate the model
def evaluate_model(model, test_dataset):
    # Evaluate your model
    evaluation_results = model.evaluate(test_dataset)
    return evaluation_results

# Step 6: Test the model
def test_model(model, test_images):
    # Test your model
    text_regions = model.predict(test_images)
    return text_regions

# Specify the number of classes for your task
num_classes = 2

# Specify the learning rate and number of epochs for training
learning_rate = 0.08
epochs = 2

# Step 1: Build the Mask R-CNN model
model = build_mask_rcnn_model(num_classes)

# Step 2: Create the training pipeline
create_training_pipeline(model, learning_rate)

# Step 3: Train the model
train_model(model, train_dataset, epochs)

# Step 4: Evaluate the model
evaluation_results = evaluate_model(model, test_dataset)

# Step 6: Test the model
test_images = r'C:\Users\Admin\OneDrive\Desktop\New folder (2)\130.png'
text_regions = test_model(model, test_images)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_7_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: image (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext]